In [1]:
import pandas as pd

# Загружаем датасетыс сырыми данными

Исходные датасеты должны иметь столбцы MotionNumber, TouchNumer, XCoordinate, YCoordinate, Size, Time, Type и представлять собой csv файл с ; в качестве разделителя.
Датасет можно получить с помощью приложения TouchRecognitor:
https://github.com/AllFi/TouchRecognitor

In [2]:
data_mine = pd.read_csv('datasets/FILIPPOV_ALEKSANDR.csv', index_col = None, sep=";")
data_mine2 = pd.read_csv('datasets/FILIPPOV_ALEKSANDR_2.csv', index_col = None, sep=";")
data_test1 = pd.read_csv('datasets/KALENIK_DANIL.csv', index_col = None, sep=";")
data_test2 = pd.read_csv('datasets/SKORODUMOV_ANATOLIJ.csv', index_col = None, sep=";")
data_test3 = pd.read_csv('datasets/BAKAJ_ZhAMGYRChIEV.csv', index_col = None, sep=";")
data_test4 = pd.read_csv('datasets/DMITRIJ_LAPShIN.csv', index_col = None, sep=";")
data_test5 = pd.read_csv('datasets/VIKTOR_TARASOV.csv', index_col = None, sep=";")

# Создаем новый датафрейм для признаков

В этом модуле мы непосредственно формируем датасет с признаками. Пока оставим такую же структуру. Чтобы добавить новый признак
необходимо написать функцию типа add_feature_feature_name и вызвать её из функции ниже.

In [4]:
def create_features_vec_num(data):
    df = pd.DataFrame(index=range(data.iloc[data.shape[0] - 1][0] + 1), columns=['MotionId','Type'])
    for i in range(data.iloc[data.shape[0] - 1][0]):
        try:
            dat = data[(data.MotionNumber == i )]
            df.set_value(i,1, dat.get_value(0,6,takeable=True), takeable=True)
            df.set_value(i,0, i, takeable=True)
        except:
            continue

    df = add_feature_vec(df, data)
    df = add_feature_time(df,data)
    df = add_feature_distance(df,data)
    df = add_feature_avrg_speed(df,data)
    df = add_feature_avrg_size(df,data)
    df = df.fillna(method='pad')
    return df

In [5]:
def add_feature_avrg_size(df_features, data):
    df_features.insert(df_features.shape[1],'AvrgSize', None)
    for i in range(df_features.shape[0]):
        sum_size = 0 
        dataMN = data[(data.MotionNumber == i )]
        for j in range(dataMN.shape[0]):
            try:
                sum_size += dataMN[["Size"]].get_value(j,0, takeable=True)
            except:
                continue
        try:
            avrg_size = sum_size / dataMN.shape[0]
            
        except:
            avrg_size = None
        df_features.set_value(i,df_features.shape[1] - 1,avrg_size, takeable=True)
    df_features = df_features.fillna(method="pad")
    return df_features

In [6]:
def add_feature_avrg_speed(df_features, data):
    df_features.insert(df_features.shape[1],'AvrgSpeed', None)
    for i in range(df_features.shape[0]):
        try:
            avrg_speed = df_features[["Distance"]].get_value(i,0, takeable=True) / df_features[["Time"]].get_value(i,0, takeable=True) 
        except:
            continue
        df_features.set_value(i,df_features.shape[1] - 1,avrg_speed, takeable=True)
    df_features = df_features.fillna(method="pad")
    return df_features

In [7]:
def add_feature_distance(df_features, data):
    df_features.insert(df_features.shape[1],'Distance', None)
    for i in range(data.iloc[data.shape[0] - 1][0]):
        dataMN = data[(data.MotionNumber == i )]
        dat = dataMN[(dataMN.TouchNumber == 0) ]
        dat2 = dataMN[(dataMN.TouchNumber == 1)]
        distance1 = 0
        distance2 = 0
        for j in range(dat.shape[0]-1):
            try:
                x_disp = dat[["XCoordinate"]].get_value(j+1,0, takeable=True) - dat[["XCoordinate"]].get_value(j,0, takeable=True)
                y_disp = dat[["YCoordinate"]].get_value(j+1,0, takeable=True) - dat[["YCoordinate"]].get_value(j,0, takeable=True)
                distance1 += (x_disp**2 + y_disp**2)**(1/2)
            except:
                continue
            if dat2.shape[0] != 0:
                try:
                    x_disp = dat2[["XCoordinate"]].get_value(j+1,0, takeable=True) - dat2[["XCoordinate"]].get_value(j,0, takeable=True)
                    y_disp = dat2[["YCoordinate"]].get_value(j+1,0, takeable=True) - dat2[["YCoordinate"]].get_value(j,0, takeable=True)
                    distance2 += (x_disp**2 + y_disp**2)**(1/2)
                except:
                    continue
            distance = distance1 + distance2
            df_features.set_value(i,df_features.shape[1] - 1,distance, takeable=True)
    df_features = df_features.fillna(method="pad")
    return df_features

In [8]:
def add_feature_time(df_features, data):
    df_features.insert(df_features.shape[1],'Time', None)
    for i in range(data.iloc[data.shape[0] - 1][0]):
        dataMN = data[(data.MotionNumber == i )]
        time = dataMN.Time.max() - dataMN.Time.min()
        df_features.set_value(i,df_features.shape[1] - 1,time, takeable=True)
    df_features = df_features.fillna(method="pad")
    return df_features

In [9]:
def add_feature_vec(df_features, data):
    df_features.insert(df_features.shape[1],'XVec', None)
    df_features.insert(df_features.shape[1],'YVec', None)
    
    for i in range(data.iloc[data.shape[0] - 1][0]):
        #print(data[(data.MotionNumber == i )][(data.Time == data.Time.max())]
        dataMN = data[(data.MotionNumber == i )]
        dat = dataMN[(dataMN.TouchNumber == 0) ]
        dat2 = dataMN[(dataMN.TouchNumber == 1)]
        try:
            XVec = dat[dat.Time==dat.Time.max()][["XCoordinate"]].get_value(0,0, takeable=True) - dat[dat.Time==dat.Time.min()][["XCoordinate"]].get_value(0,0, takeable=True)
            YVec = dat[dat.Time==dat.Time.max()][["YCoordinate"]].get_value(0,0, takeable=True) - dat[dat.Time==dat.Time.min()][["YCoordinate"]].get_value(0,0, takeable=True)
        except:
            continue
        if dat2.shape[0] != 0:
            XVec2 = dat2[dat2.Time==dat2.Time.max()][["XCoordinate"]].get_value(0,0, takeable=True) - dat2[dat2.Time==dat2.Time.min()][["XCoordinate"]].get_value(0,0, takeable=True)
            YVec2 = dat2[dat2.Time==dat2.Time.max()][["YCoordinate"]].get_value(0,0, takeable=True) - dat2[dat2.Time==dat2.Time.min()][["YCoordinate"]].get_value(0,0, takeable=True)
            if dat2.YCoordinate.max() > dat.YCoordinate.max(): 
                XVec = XVec2 - XVec
                YVec = YVec2 - YVec
            else:
                XVec = XVec - XVec2
                YVec = YVec - YVec2
                
        df_features.set_value(i,df_features.shape[1] - 2,XVec, takeable=True)
        df_features.set_value(i,df_features.shape[1] - 1,YVec, takeable=True)
    df_features = df_features.fillna(method="pad")
    return df_features

# Функция отображения объектов на плоскости по двум признакам

In [10]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_2d(df, feature_x, feature_y):
    X = df[[feature_x, feature_y, "Type"]]

    X = X.fillna(0)
    #print(X)
    colors = ['red', 'blue', 'lightgreen', 'gray', 'cyan', "green", "black", "yellow", "pink"]
    plt.figure(figsize=(16,16))
    plt.scatter(x=X[[feature_x]], y=X[[feature_y]], facecolors='none', edgecolors='black',
                       alpha=1.0, linewidth=1, marker='o',
                       s=55, label='test_set')

    i = 0;
    for t in ["DU", "UD", "LR", "RL", "ZI", "TP", "ZO", "TC", "TCC"]:
        plt.scatter(x=X[X.Type == t][[feature_x]], y=X[X.Type == t][[feature_y]], facecolors=colors[i], edgecolors='black',
                       alpha=1.0, linewidth=1, marker='o',
                       s=55, label='test_set')
        i += 1;
    plt.show()

In [11]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_compare_2d(df1, df2, feature1 ,feature2, ttype):
    X1 = df1[[feature1, feature2, "Type"]]
    X1 = X1.fillna(0)
    
    X2 = df2[[feature1, feature2, "Type"]]
    X2 = X2.fillna(0)
    
    colors = ['red', 'blue', 'lightgreen', 'gray', 'cyan', "green", "black", "yellow", "pink"]
    plt.figure(figsize=(16,16))
    plt.scatter(x=X1[X1.Type == ttype][[feature1]], y=X1[X1.Type == ttype][[feature2]], facecolors='red', edgecolors='black',
                       alpha=1.0, linewidth=1, marker='o',
                       s=55, label='test_set')
    
    plt.scatter(x=X2[X2.Type == ttype][[feature1]], y=X2[X2.Type == ttype][[feature2]], facecolors='blue', edgecolors='black',
                       alpha=1.0, linewidth=1, marker='o',
                       s=55, label='test_set')
    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.title("Feature Type:" + ttype)
    plt.show()

# Функция отображения объектов в пространстве по трем признакам


In [12]:
from mpl_toolkits.mplot3d import Axes3D

In [13]:
def plot_3d(df, feature_x, feature_y, feature_z):
    X = df[[feature_x, feature_y, feature_z, "Type"]]

    X = X.fillna(0)
    #print(X)
    colors = ['red', 'blue', 'lightgreen', 'gray', 'cyan', "green", "black", "yellow", "pink"]
    fig = plt.figure(figsize=(16,16))
    ax = fig.add_subplot(111, projection='3d')
    
    i = 0;
    for t in ["DU", "UD", "LR", "RL", "ZI", "TP", "ZO", "TC", "TCC"]:
        ax.scatter(xs=X[X.Type == t][[feature_x]], ys=X[X.Type == t][[feature_y]], zs=X[X.Type == t][[feature_z]],  c=colors[i], s=55)
        i += 1;
    plt.show()

# Функция для получения датасета для одного конкретного типа движений

In [14]:
def extract_data(data, target):
    return data[data.Type==target]


In [15]:
def combine_data(df_list):
    df = df_list[0].copy()
    for i in range(1,len(df_list)):
        df = df.append(df_list[i],ignore_index=True)
    return df
        

# Поиск аномалий

In [16]:
df = create_features_vec_num(data_mine)
X_train = extract_data(df, "DU")

In [17]:
df = create_features_vec_num(data_mine2)
X_inline = extract_data(df, "DU")

In [20]:
df = create_features_vec_num(data_test1)
X_test1 = extract_data(df, "DU")
df = create_features_vec_num(data_test2)
X_test2 = extract_data(df, "DU")
df = create_features_vec_num(data_test3)
X_test3 = extract_data(df, "DU")
df = create_features_vec_num(data_test4)
X_test4 = extract_data(df, "DU")
df = create_features_vec_num(data_test5)
X_test5 = extract_data(df, "DU")

In [21]:
df_list = [X_test1, X_test2, X_test3, X_test4, X_test5]
X_outline = combine_data(df_list)

In [22]:
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

In [ ]:
for i in range(10):
    print(i)
    isolation_forest = IsolationForest(random_state=0, n_estimators=(i+1)*10,  n_jobs=8)
    X_tr = X_train.iloc[:,[2,3,4,5,6,7]]
    isolation_forest.fit(X_tr.as_matrix())

    X_tst = X_outline.iloc[:,[2,3,4,5,6,7]]
    outliers_count = 0
    for outlier in X_tst.as_matrix():
        outlier = outlier.reshape(1, -1)
        if isolation_forest.predict(outlier) == -1:
            outliers_count+=1
    print("FAR=", 100 - (outliers_count * 100) /len(X_tst), "%")

    X_tst = X_inline.iloc[:,[2,3,4,5,6,7]]
    outliers_count = 0
    for outlier in X_tst.as_matrix():
        outlier = outlier.reshape(1, -1)
        if isolation_forest.predict(outlier) == -1:
            outliers_count+=1
    print("FRR=", (outliers_count * 100) /len(X_tst), "%")

0
FAR= 24.0 %
FRR= 8.6 %
1
FAR= 17.444444444444443 %
FRR= 11.3 %
2
FAR= 18.055555555555557 %
FRR= 11.1 %
3
FAR= 15.944444444444443 %
FRR= 10.1 %
4
FAR= 15.944444444444443 %
FRR= 9.9 %
5
FAR= 18.444444444444443 %
FRR= 9.8 %
6


In [ ]:
FAR = 15,78 FRR = 11,5